In [1]:
import cv2
import tensorflow as tf
import numpy as np
import math
model = tf.keras.models.load_model("05resnet.model")

In [9]:
def prepare(ima):
    IMG_SIZE = 100# 50 in txt-based
    img_array=ima*255
    #img_array = cv2.cvtColor(ima,cv2.COLOR_BGR2GRAY)
    img_array=img_array/255.0  
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1,IMG_SIZE, IMG_SIZE,1)

def paa_skin(image,image1):  
    new1 = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    dist = cv2.distanceTransform(new1, cv2.DIST_L2, 5)
    cv2.normalize(dist, dist, 0, 1.0, cv2.NORM_MINMAX)
    return image1,dist

def alpha(prediction):
    if(prediction==0):
        prediction='A'
    elif(prediction==1):
        prediction='B'
    elif(prediction==2):
        prediction='C'
    elif(prediction==3):
        prediction='D'
    elif(prediction==4):
        prediction='E'
    elif(prediction==5):
        prediction='F'
    elif(prediction==6):
        prediction='G'
    elif(prediction==7):
        prediction='H'
    elif(prediction==8):
        prediction='I'
    elif(prediction==9):
        prediction='J'
    elif(prediction==10):
        prediction='K'
    elif(prediction==11):
        prediction='L'
    elif(prediction==12):
        prediction='M'
    elif(prediction==13):
        prediction='N'
    elif(prediction==14):
        prediction='O'
    elif(prediction==15):
        prediction='P'
    elif(prediction==16):
        prediction='Q'
    elif(prediction==17):
        prediction='R'
    elif(prediction==18):
        prediction='S'
    elif(prediction==19):
        prediction='T'
    elif(prediction==20):
        prediction='U'
    elif(prediction==21):
        prediction='V'
    elif(prediction==22):
        prediction='W'
    elif(prediction==23):
        prediction='X'
    elif(prediction==24):
        prediction='Y'
    elif(prediction==25):
        prediction='Z'
    else:
        prediction='nothing'
    return prediction

def callback(x):
    pass

In [ ]:
cv2.namedWindow('image1')
cx=100
cy=100
rw=300
rh=300
cap=cv2.VideoCapture(0)
ilowH = 0
ihighH = 255
ilowS = 0
ihighS = 255
ilowV = 0
ihighV = 255
thresh1=100
thresh2=1
cv2.createTrackbar('lowH1', 'image1', ilowH, 179, callback)
cv2.createTrackbar('highH1', 'image1', ihighH, 179, callback)
cv2.createTrackbar('lowS1', 'image1', ilowS, 255, callback)
cv2.createTrackbar('highS1', 'image1', ihighS, 255, callback)
cv2.createTrackbar('lowV1', 'image1', ilowV, 255, callback)
cv2.createTrackbar('highV1', 'image1', ihighV, 255, callback)
cv2.createTrackbar('thresh1','image1',thresh1,255,callback)
cv2.createTrackbar('thresh2','image1',thresh2,255,callback)
while True:
    try:
        ret,frame=cap.read()
        ilowH1 = cv2.getTrackbarPos('lowH1', 'image1')
        ihighH1 = cv2.getTrackbarPos('highH1', 'image1')
        ilowS1 = cv2.getTrackbarPos('lowS1', 'image1')
        ihighS1 = cv2.getTrackbarPos('highS1', 'image1')
        ilowV1 = cv2.getTrackbarPos('lowV1', 'image1')
        ihighV1 = cv2.getTrackbarPos('highV1', 'image1')
        thresh1 = cv2.getTrackbarPos('thresh1', 'image1')
        thresh2 = cv2.getTrackbarPos('thresh2', 'image1')        
        hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
        low1=np.array([ilowH1,ilowS1,ilowV1])
        high1=np.array([ihighH1,ihighS1,ihighV1])
        img_mask1=cv2.inRange(hsv,low1,high1)
        output1=cv2.bitwise_and(frame,frame,mask=img_mask1)
        pre=output1[cx:rw,cy:rh]
        pre=cv2.GaussianBlur(pre,(5,5),0)
        pre=cv2.cvtColor(pre,cv2.COLOR_BGR2GRAY)
        pre=np.uint8(pre)
        edged=cv2.Canny(pre,thresh1,thresh2)
        prediction = model.predict([prepare(edged)])
        prediction=np.argmax(prediction)
        x1=alpha(prediction)
        cv2.putText(frame,x1,(60,80),cv2.FONT_HERSHEY_SIMPLEX,3.0,(255,255,255),lineType=cv2.LINE_AA)
        cv2.rectangle(frame,(cx,cy),(cx+rw,cy+rh),(255,255,255),5)
        cv2.putText(output1,x1,(60,80),cv2.FONT_HERSHEY_SIMPLEX,3.0,(255,255,255),lineType=cv2.LINE_AA)
        cv2.rectangle(output1,(cx,cy),(cx+rw,cy+rh),(255,255,255),5)
        cv2.imshow("image",output1)
        cv2.imshow("image1",frame)
        cv2.imshow("image2",edged)
    except:
        pass
    if (cv2.waitKey(1) == 13):
            break
cv2.destroyAllWindows()
cap.release()